## Parte 1: Importações e Configurações

In [17]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
import time
import json

# Configuração de dispositivo (GPU é recomendada)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Semente para reprodutibilidade
torch.manual_seed(42)
np.random.seed(42)

Usando dispositivo: cpu


## Parte 2: Discretização (Grades + Clusters)

In [6]:
class GlobalGridDiscretizer:
    def __init__(self, cell_size_meters=100, n_clusters=3):
        self.cell_size = cell_size_meters
        self.n_clusters = n_clusters
        self.grids = {} 
        self.token_mapper = LabelEncoder()
        self.is_fitted = False

    def _get_grid_params(self, lat_min, lat_max, lat_mean):
        # Conversão de metros para graus (aproximação local)
        # 1 grau Lat ~= 111.111 km
        lat_step = self.cell_size / 111111.0
        # 1 grau Lon ~= 111.111 * cos(lat) km
        lon_step = self.cell_size / (111111.0 * np.cos(np.radians(lat_mean)))
        n_rows = math.ceil((lat_max - lat_min) / lat_step)
        return lat_step, lon_step, n_rows

    def fit(self, df):
        """Identifica 3 regiões fixas e cria os vocabulários."""
        print(f"1. Identificando {self.n_clusters} regiões (K-Means)...")
        
        # Coleta pontos de amostragem (Início e Fim da trajetória)
        # Isso garante que a região cubra toda a rota
        points = []
        for _, row in df.iterrows():
            points.append([row['path_lat'][0], row['path_lon'][0]]) # Início
            points.append([row['path_lat'][-1], row['path_lon'][-1]]) # Fim
        
        points = np.array(points)
        
        # --- ALTERAÇÃO AQUI: K-MEANS ---
        # Como as regiões são globalmente distantes, a distância Euclidiana 
        # simples (sem Haversine) é suficiente para separar os grupos macro.
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(points)
        
        unique_labels = set(labels)
        
        # Configuração das Grades
        global_offset = 1 # Token 0 reservado para PAD
        
        for label in unique_labels:
            mask = labels == label
            cluster_points = points[mask]
            
            # Define o Bounding Box (Retângulo) da região
            lat_min, lon_min = cluster_points.min(axis=0)
            lat_max, lon_max = cluster_points.max(axis=0)
            
            # Adiciona margem de segurança (buffer)
            # ~1km de margem para evitar pontos na borda exata
            buffer = 0.01 
            lat_min -= buffer; lat_max += buffer
            lon_min -= buffer; lon_max += buffer
            
            # Calcula geometria da grade
            lat_step, lon_step, n_rows = self._get_grid_params(lat_min, lat_max, (lat_min+lat_max)/2)
            n_cols = math.ceil((lon_max - lon_min) / lon_step)
            
            self.grids[label] = {
                'lat_min': lat_min, 'lon_min': lon_min,
                'lat_step': lat_step, 'lon_step': lon_step,
                'n_rows': n_rows, 'n_cols': n_cols,
                'offset': global_offset
            }
            
            # Atualiza offset para o próximo cluster não sobrepor tokens
            print(f"   Cluster {label}: Grade {n_rows}x{n_cols} criada.")
            global_offset += (n_rows * n_cols)
            
        print("2. Mapeando tokens brutos para índices compactos...")
        
        # Coleta todos os tokens reais para criar o LabelEncoder
        all_raw_tokens = []
        for _, row in df.iterrows():
            t = self._trajectory_to_raw_tokens(row['path_lat'], row['path_lon'])
            all_raw_tokens.extend(t)
            
        # Treina o compactador de IDs
        self.token_mapper.fit(np.unique(all_raw_tokens))
        
        self.vocab_size = len(self.token_mapper.classes_) + 1
        print(f"   Vocabulário Final: {self.vocab_size} células únicas visitadas.")
        self.is_fitted = True

    def _get_raw_token(self, lat, lon):
        # Verifica em qual dos 3 grids o ponto cai
        for grid in self.grids.values():
            if grid['lat_min'] <= lat < (grid['lat_min'] + grid['n_rows']*grid['lat_step']) and \
               grid['lon_min'] <= lon < (grid['lon_min'] + grid['n_cols']*grid['lon_step']):
                
                row = int((lat - grid['lat_min']) / grid['lat_step'])
                col = int((lon - grid['lon_min']) / grid['lon_step'])
                
                # Token Bruto = Offset do Cluster + Índice Linear Local
                return grid['offset'] + (row * grid['n_cols'] + col)
        return 0 # Ponto fora das 3 regiões mapeadas

    def _trajectory_to_raw_tokens(self, lats, lons):
        tokens = []
        last = None
        for lat, lon in zip(lats, lons):
            t = self._get_raw_token(lat, lon)
            # Lógica de compressão: ignora zeros e repetições consecutivas
            if t != 0 and t != last: 
                tokens.append(t)
                last = t
        return tokens

    def transform(self, lats, lons):
        """Converte Lat/Lon -> Token Mapeado (Pronto para o Modelo)"""
        raw_tokens = self._trajectory_to_raw_tokens(lats, lons)
        
        # Filtra tokens desconhecidos (caso apareçam na validação/teste)
        valid_raw = [t for t in raw_tokens if t in self.token_mapper.classes_]
        
        if not valid_raw: return [0]
        
        # Retorna IDs compactos (+1 pois 0 é PAD)
        return (self.token_mapper.transform(valid_raw) + 1).tolist()

## Parte 3: Função de Ruido (Data Augmentation)

In [7]:
def apply_gps_noise(lats, lons, noise_meters=15.0, prob=0.5):
    """Adiciona ruído gaussiano às coordenadas."""
    if np.random.rand() > prob:
        return lats, lons
    
    n = len(lats)
    noise_lat_m = np.random.normal(0, noise_meters, n)
    noise_lon_m = np.random.normal(0, noise_meters, n)
    
    # Metros -> Graus
    delta_lat = noise_lat_m / 111111.0
    avg_lat = np.radians(np.mean(lats))
    delta_lon = noise_lon_m / (111111.0 * np.cos(avg_lat))
    
    new_lats = np.array(lats) + delta_lat
    new_lons = np.array(lons) + delta_lon
    
    return new_lats.tolist(), new_lons.tolist()

## Parte 4: Preparação dos Dados

In [8]:
class TrajectoryDataset(Dataset):
    def __init__(self, df, discretizer, target_scaler, augment=False):
        self.df = df.reset_index(drop=True)
        self.discretizer = discretizer
        self.scaler = target_scaler
        self.augment = augment # Só True no treino!

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        lats, lons = row['path_lat'], row['path_lon']
        
        # 1. Aplica Ruído (Se for treino)
        if self.augment:
            lats, lons = apply_gps_noise(lats, lons)
            
        # 2. Tokenização
        tokens = self.discretizer.transform(lats, lons)
        
        # 3. Preparação do Alvo (Normalizado)
        dest = np.array([[row['dest_lat'], row['dest_lon']]])
        dest_scaled = self.scaler.transform(dest).flatten()
        
        return torch.tensor(tokens, dtype=torch.long), torch.tensor(dest_scaled, dtype=torch.float32)

# Função para agrupar batches de tamanhos diferentes (Padding)
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_stacked = torch.stack(targets)
    return inputs_padded, targets_stacked

## Parte 5: O Modelo (Encoder-Decoder com Attention)

In [9]:
class TrajectoryEncoderAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super(TrajectoryEncoderAttention, self).__init__()
        
        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. Encoder LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=False)
        
        # 3. Attention Mechanism (Bahdanau Style simplificado)
        self.attn = nn.Linear(hidden_dim, 1)
        
        # 4. Regressor Head (Decoder simples)
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 2) # Lat, Lon
        )

    def forward(self, x):
        # x: [batch, seq_len]
        
        # Mask para ignorar padding na atenção
        mask = (x != 0).unsqueeze(-1).float() # [batch, seq, 1]
        
        embed = self.embedding(x)
        output, (hidden, _) = self.lstm(embed) 
        # output: [batch, seq, hidden]
        
        # Calcular Scores de Atenção
        attn_scores = self.attn(output) # [batch, seq, 1]
        
        # Aplicar máscara (zerar score onde é padding)
        attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        attn_weights = torch.softmax(attn_scores, dim=1)
        
        # Context Vector (Soma ponderada)
        context_vector = torch.sum(attn_weights * output, dim=1) # [batch, hidden]
        
        # Predição
        coords = self.regressor(context_vector)
        
        return coords, context_vector # Retorna o vetor para usar em similaridade depois!

## Parte 6: Execução (Pipeline Principal)

In [ ]:
train = pd.read_csv('..\\te-aprendizado-de-maquina\\train.csv')

In [36]:
test = pd.read_csv('..\\te-aprendizado-de-maquina\\test.csv')
test.head()

,trajectory_id,user_id,n_points,prefix_ratio,prefix_n_points,path_lat,path_lon
0,000_20081028003826,0,1477,0.3,443,"[40.01229, 40.012594, 40.012581, 40.012453, 40...","[116.297072, 116.297269, 116.297263, 116.29717..."
1,000_20081112023003,0,681,0.3,204,"[39.995805, 39.995833, 39.995981, 39.995929, 3...","[116.326236, 116.326268, 116.32611, 116.326169..."
2,000_20081118095400,0,254,0.3,76,"[40.010603, 40.010601, 40.010586, 40.010692, 4...","[116.322627, 116.322636, 116.322561, 116.32261..."
3,000_20081119112035,0,419,0.3,125,"[40.001038, 40.000522, 40.000638, 40.000662, 4...","[116.324874, 116.324638, 116.324694, 116.32472..."
4,000_20081212042525,0,338,0.3,101,"[40.007451, 40.00743, 40.008937, 40.008948, 40...","[116.323341, 116.32339, 116.321548, 116.321455..."


In [20]:
train.head()

,trajectory_id,user_id,n_points,prefix_ratio,prefix_n_points,path_lat,path_lon,dest_lat,dest_lon
0,000_20081023025304,0,908,0.3,272,"[39.984702, 39.984683, 39.984686, 39.984688, 3...","[116.318417, 116.31845, 116.318417, 116.318385...",40.009328,116.320887
1,000_20081024020959,0,244,0.3,73,"[40.008304, 40.008413, 40.007171, 40.007209, 4...","[116.319876, 116.319962, 116.319458, 116.31948...",40.009209,116.321162
2,000_20081026134407,0,745,0.3,223,"[39.907414, 39.907374, 39.907027, 39.907006, 3...","[116.370017, 116.370074, 116.37036, 116.370415...",39.926426,116.320399
3,000_20081029093038,0,182,0.3,54,"[39.991364, 39.991551, 39.991821, 39.991771, 3...","[116.326605, 116.326653, 116.326695, 116.32667...",39.966701,116.327688
4,000_20081103232153,0,2231,0.3,669,"[39.996948, 39.996849, 39.994409, 39.995076, 3...","[116.325747, 116.325763, 116.326968, 116.32662...",39.999659,116.324747


In [23]:
def processamento(df):
    feats = []
    for _, row in df.iterrows():
        lats = json.loads(row['path_lat'])
        lons = json.loads(row['path_lon'])

        dest_lat = float(row['dest_lat'])
        dest_lon = float(row['dest_lon'])

        feats.append({
            "path_lat": lats,
            "path_lon": lons,
            "dest_lat": dest_lat,
            "dest_lon": dest_lon
        })

    return pd.DataFrame(feats)

In [29]:
train = processamento(train)


In [30]:
# --- 1. SETUP DOS DADOS ---
# df_train deve ter colunas: 'path_lat', 'path_lon' (listas) e 'dest_lat', 'dest_lon' (floats)
# Vamos assumir que 'train' é o seu dataframe carregado

# A. Treinar o Discretizador
discretizer = GlobalGridDiscretizer(cell_size_meters=200) # Células maiores = Menos tokens = Treino mais rápido
discretizer.fit(train) 

# B. Treinar o Scaler de Destino
target_scaler = MinMaxScaler(feature_range=(-1, 1))
target_scaler.fit(train[['dest_lat', 'dest_lon']].values)

# C. Dividir Treino/Validação
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

# D. Criar Datasets e Loaders
# Augment=True no treino para robustez a ruído!
ds_train = TrajectoryDataset(train_df, discretizer, target_scaler, augment=True)
ds_val = TrajectoryDataset(val_df, discretizer, target_scaler, augment=False)

dl_train = DataLoader(ds_train, batch_size=64, shuffle=True, collate_fn=collate_fn)
dl_val = DataLoader(ds_val, batch_size=64, shuffle=False, collate_fn=collate_fn)

# --- 2. SETUP DO MODELO ---
model = TrajectoryEncoderAttention(
    vocab_size=discretizer.vocab_size, 
    embedding_dim=64, 
    hidden_dim=128
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# --- 3. LOOP DE TREINO ---
print("\nIniciando Treinamento...")
history = []

for epoch in range(10): # Teste com 10 épocas
    model.train()
    total_loss = 0
    
    for x, y in dl_train:
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        pred, _ = model(x) # Ignora o vetor de contexto no treino
        
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    avg_loss = total_loss / len(dl_train)
    print(f"Epoch {epoch+1} | Loss: {avg_loss:.5f}")

print("Treino Concluído!")

# --- 4. TESTE E RECUPERAÇÃO DE EMBEDDINGS ---
model.eval()
x_sample, y_sample = next(iter(dl_val))
x_sample = x_sample.to(device)

with torch.no_grad():
    preds, embeddings = model(x_sample)
    
    # Inverter escala
    preds_real = target_scaler.inverse_transform(preds.cpu().numpy())
    target_real = target_scaler.inverse_transform(y_sample.numpy())

# Mostra erro em metros do primeiro exemplo
dist = np.linalg.norm(preds_real[0] - target_real[0]) * 111111 # Aprox grosseira
print(f"\nExemplo de Erro: ~{dist:.0f} metros")
print(f"Vetor de Similaridade gerado (tamanho): {embeddings.shape}")

1. Identificando 3 regiões (K-Means)...
   Cluster 0: Grade 27862x29773 criada.
   Cluster 1: Grade 16555x37249 criada.
   Cluster 2: Grade 11069x10237 criada.
2. Mapeando tokens brutos para índices compactos...
   Vocabulário Final: 154505 células únicas visitadas.

Iniciando Treinamento...
Epoch 1 | Loss: 0.03410
Epoch 2 | Loss: 0.01356
Epoch 3 | Loss: 0.01170
Epoch 4 | Loss: 0.00892
Epoch 5 | Loss: 0.00621
Epoch 6 | Loss: 0.00349
Epoch 7 | Loss: 0.00204
Epoch 8 | Loss: 0.00129
Epoch 9 | Loss: 0.00101
Epoch 10 | Loss: 0.00100
Treino Concluído!

Exemplo de Erro: ~136280 metros
Vetor de Similaridade gerado (tamanho): torch.Size([64, 128])


In [32]:
def processamento_test(df):
    feats = []
    for _, row in df.iterrows():
        lats = json.loads(row['path_lat'])
        lons = json.loads(row['path_lon'])

        feats.append({
            "path_lat": lats,
            "path_lon": lons
        })

    return pd.DataFrame(feats)

In [37]:
test = processamento_test(test)
test.head()

,path_lat,path_lon
0,"[40.01229, 40.012594, 40.012581, 40.012453, 40...","[116.297072, 116.297269, 116.297263, 116.29717..."
1,"[39.995805, 39.995833, 39.995981, 39.995929, 3...","[116.326236, 116.326268, 116.32611, 116.326169..."
2,"[40.010603, 40.010601, 40.010586, 40.010692, 4...","[116.322627, 116.322636, 116.322561, 116.32261..."
3,"[40.001038, 40.000522, 40.000638, 40.000662, 4...","[116.324874, 116.324638, 116.324694, 116.32472..."
4,"[40.007451, 40.00743, 40.008937, 40.008948, 40...","[116.323341, 116.32339, 116.321548, 116.321455..."


In [ ]:
def prever_destino(path_lats, path_lons, model, discretizer, scaler):
    """
    Recebe uma trajetória bruta e retorna a previsão (Lat, Lon).
    """
    model.eval() # Modo de avaliação (desliga dropout)
    
    # 1. Pré-processamento (Igual ao treino)
    # Transforma Lat/Lon -> Tokens
    tokens = discretizer.transform(path_lats, path_lons)
    
    # Transforma em Tensor e adiciona dimensão de batch (Batch Size = 1)
    # Shape: [1, seq_len]
    tensor_x = torch.tensor([tokens], dtype=torch.long).to(device)
    
    # 2. Inferência
    with torch.no_grad():
        # O modelo retorna (Predição, Contexto). Queremos só a predição [0]
        preds_scaled, _ = model(tensor_x)
    
    # 3. Pós-processamento (Inverter a escala)
    # Transforma [-0.5, 0.8] de volta para [39.9, 116.4]
    preds_real = scaler.inverse_transform(preds_scaled.cpu().numpy())
    
    return preds_real[0] # Retorna [lat, lon]

# --- Teste Rápido ---
# Pegando um exemplo do conjunto de validação
idx = 0
exemplo = val_raw.iloc[idx] # Usando o dataframe original de validação
path_lat = exemplo['path_lat']
path_lon = exemplo['path_lon']

pred_lat, pred_lon = prever_destino(path_lat, path_lon, model, discretizer, target_scaler)

print(f"Trajetória com {len(path_lat)} pontos.")
print(f"Destino Real:     [{exemplo['dest_lat']:.5f}, {exemplo['dest_lon']:.5f}]")
print(f"Destino Previsto: [{pred_lat:.5f}, {pred_lon:.5f}]")

Dataset de Teste pronto: 3013 trajetórias.
Iniciando inferência no Teste...


KeyError: 'dest_lat'